In [2]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities
from setups import UIdisplay

style = {"description_width": "initial"}

In [2]:
### Create CASE folder for power/thermal analysis ###
CASE = widgets.Text(value="TEST", description="CASE Name:", style=style, disabled=False)
PWfc = FileChooser("./")   ### power file generated from FC
PWType = widgets.RadioButtons(options=["IPF","CSV","TILE"], description="Power file type", disabled=False)
HB1 = HBox([Label("Select Power File:"), PWfc, PWType])
VBox([CASE, HB1])

In [3]:
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)

In [3]:
### Optional: Manually given the design area ###
isManually = widgets.Select(options=["True", "False"], value="False", style=style, 
                            description="Design Area in Manually", disabled=False)
llx = widgets.FloatText(value=0.0, description="LLX", disabled=False)
lly = widgets.FloatText(value=0.0, description="LLY", disabled=False)
urx = widgets.FloatText(value=0.0, description="URX", disabled=False)
ury = widgets.FloatText(value=0.0, description="URY", disabled=False)
HB1 = HBox([llx, lly, urx, ury])
VBox([isManually, HB1])

In [10]:
import importlib
importlib.reload(FCpowermap)

designArea = []
try:
    if isManually.value in ["True"]:
        designArea = [llx.value, lly.value, urx.value, ury.value]
except:
    pass

FCpwrView = FCpowermap.FCPowerView(PWfc.selected, PWType.value, DesignArea=designArea)

#cells = ["i0mfvp803mc1q03x5","i0mfuz443aa1q06x5","i0mfuz803mc1q06x5","i0moai013ab1n09x5","i0moai013ac1d12x5"]
#cells = ["i0mfuz443aa1q06x5"] #["i0maoi302aa1d09x5"] #["i0mfuz443aa1q06x5"]
#FCpwrView.modifyCSV(modifyCells=cells, pwrFactor=50)
#FCpwrView.modifyCSV(modifyCells="ALL", pwrFactor=0.000001)

resolution = widgets.IntText(value=5, description="Tile Resolution(um):", style=style, disabled=False)
topN = widgets.IntText(value=10, description="Top-N High Power Tiles:", style=style, disabled=False)
VBox([resolution, topN])

<INFO> Instances Area:30.49,42.62,102.06,113.08; Total PWR=18.409mW


In [21]:
import importlib
importlib.reload(UIdisplay)

FCpwrView.translate2TilePwr(resolution=resolution.value)
candidatedCells = FCpwrView.TilePwrAnalysis(resolution=resolution.value, topN=topN.value)

### select the candidated cells, droplist: "cell,contributed_power_in_ratio" ###
droplist = []
for c in candidatedCells:
    _str = "{}, {}%".format(c[0], c[2])
    droplist.append(_str)

dumpCells = []
out = UIdisplay.askDropItems2Processing("Select candidated cells", droplist, dumpCells, dropLabel="Cells:")
display(out)

In [6]:
import importlib
importlib.reload(UIdisplay)

XX, YY = 9, 9
grid = UIdisplay.createArrayPanel(XX, YY)
TSize = widgets.IntText(value=5, description="Tile Size(um):", style=style, disabled=False)
VBox([grid, TSize])

In [29]:
grid[2, 4].value

8.0

In [8]:
import importlib
importlib.reload(UIdisplay)

UIdisplay.ArrayPanel2TileIPF(XX, YY, TSize.value, grid)

BBOX: (0.0,0.0)-(90,90), Total Power: 3.0 mW


([0.0, 0.0, 90, 90], './tileIPF.csv')